In [1]:
import pandas as pd
import os
from data_cleaning import normalize_tracking, feature_engineering,get_position_count,label_run_or_pass
from data_loading import load_tracking_data, pivot_data,load_previous_year_data, load_stadium_data, load_weather_data, aggregate_data
root_dir = os.getcwd()

games_fname= os.path.join(root_dir, "data/games.csv")
plays_fname = os.path.join(root_dir,  "data/plays.csv")
players_fname = os.path.join(root_dir,  "data/players.csv")
player_plays_fname = os.path.join(root_dir,  "data/player_play.csv")
tracking_fname_list_train = [os.path.join(root_dir,  f"data/tracking_week_{i}.csv") for i in range(1,9)]

## Load merged data preprocessing for our '21 data incorporation

In [13]:
df_games=pd.read_csv(games_fname)
df_games=pd.merge(df_games, load_stadium_data(),left_on='gameId', right_on='old_game_id', how='left')
df_games=pd.merge(df_games, load_weather_data(),on='gameId', how='left')

df_plays = feature_engineering(pd.read_csv(plays_fname))

2022 done.
Downcasting floats.


In [ ]:
df_tracking = pd.concat(
        [load_tracking_data(tracking_fname) for tracking_fname in tracking_fname_list_train]
    )
df_tracking=df_tracking[['gameId', 'playId', 'nflId','club' ,'o_standard', 'dir_standard', 'x_standard', 'y_standard', 's', 'a', 'dis']].groupby(['gameId', 'playId', 'nflId','club']).agg({'s':[ 'max',],'a':[ 'max'], 
    'o_standard':['mean', 'std'],'dis':['sum'],'dir_standard':['mean', 'std'], 'x_standard':['mean', 'std'], 'y_standard':['mean', 'std']}).reset_index()
df_tracking.columns=df_tracking.columns.map('|'.join).str.strip('|')
df_players = pd.read_csv(players_fname)
df_player_plays=pd.read_csv(player_plays_fname)
df_player_plays=pd.merge(df_player_plays, load_previous_year_data(2021), left_on='teamAbbr', right_on='team_abbr', how='outer')
    # aggregate plays, tracking, players tables
df_agg1=pd.merge(pd.merge(df_tracking, df_plays, left_on=[ 'gameId', 'playId','club'], right_on=[ 'gameId', 'playId', 'possessionTeam'], how='inner'), df_player_plays, left_on=[ 'gameId', 'playId', 'nflId'], right_on=[ 'gameId', 'playId', 'nflId'], how='left' )
df_agg2=pd.merge(df_agg1, df_games, on='gameId', how='inner')
df_final=pivot_data(label_run_or_pass(get_position_count(pd.merge(df_agg2, df_players, on='nflId', how='inner'))))
merged_id_df = df_final[['gameId','playId']].drop_duplicates()

In [4]:
#merged_id_df.to_csv('data/merged_ids.csv',index=False)

# Inspect normal output of aggregate_data

We follow from 'merged_id_df' just to see what our column order should be:

In [7]:
merged_id_df = pd.read_csv('data/merged_ids.csv')

In [9]:
from data_loading import load_ftn

In [10]:
%%capture
ftn_merged = load_ftn()

In [11]:
xp_fname = os.path.join(root_dir, "data/team_xpass_ratios.csv")
pr_fname = os.path.join(root_dir, "data/team_pass_rates.csv")
cf_fname = os.path.join(root_dir, "data/coverages_faced.csv")
cu_fname = os.path.join(root_dir, 'data/coverages_used.csv')
inj_fname = os.path.join(root_dir, 'data/snaps_lost_injury.csv')

In [14]:
ftn_merged=load_ftn()
xp_df = pd.read_csv(xp_fname).drop(columns='Unnamed: 0')
pr_df = pd.read_csv(pr_fname).drop(columns='Unnamed: 0')
merged_base = merged_id_df.merge(ftn_merged,how='left',on=['gameId','playId'])
merged_base = merged_base.merge(pr_df,how='left',on=['gameId','playId'])
merged_base = merged_base.merge(xp_df,how='left',on=['gameId','playId'])
merged_base = merged_base.merge(df_games[['gameId','week']].drop_duplicates(),how='left',on=['gameId'])
merged_base = merged_base.merge(df_plays[['gameId','playId',
                                          'possessionTeam','defensiveTeam']].drop_duplicates(),
                                how='left',on=['gameId','playId'])
cf_df = pd.read_csv(cf_fname)
cu_df = pd.read_csv(cu_fname)
merged_base = merged_base.merge(cf_df,how='left',on=['possessionTeam','week'])
merged_base = merged_base.merge(cu_df,how='left',on=['defensiveTeam','week'])

inj_df = pd.read_csv(inj_fname)
merged_base = merged_base.merge(inj_df.drop(columns=['def_snaps_lost']),how='left',
                   left_on=['possessionTeam','week'], right_on=['club_code','week']).drop(columns=['club_code'])
merged_base = merged_base.merge(inj_df.drop(columns=['off_snaps_lost']),how='left',
                   left_on=['defensiveTeam','week'], right_on=['club_code','week']).drop(columns=['club_code'])

    

2022 done.
Downcasting floats.
Downcasting floats.


/Users/ccollins/Projects/NFL_BDB2025/data_loading.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pbp_ids['play_id'] = pbp_ids['play_id'].astype(int)
/Users/ccollins/Projects/NFL_BDB2025/data_loading.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pbp_ids['game_id'] = pbp_ids['game_id'].astype(str)


In [16]:
merged_base.columns

Index(['gameId', 'playId', 'n_offense_backfield', 'n_defense_box',
       'is_no_huddle', 'is_motion', 'pass_rate_off', 'pass_rate_def',
       'off_xpass', 'def_xpass', 'week', 'possessionTeam', 'defensiveTeam',
       'cover_3_off', 'cover_6_off', 'cover_1_off', 'Quarters_off',
       'Cover-2_off', 'Cover-0_off', 'Man_off', 'Other_off', 'Zone_off',
       'cover_3_def', 'cover_6_def', 'cover_1_def', 'Quarters_def',
       'Cover-2_def', 'Cover-0_def', 'Man_def', 'Other_def', 'Zone_def',
       'off_snaps_lost', 'def_snaps_lost'],
      dtype='object')

In [17]:
merged_base.head(3)

,gameId,playId,n_offense_backfield,n_defense_box,is_no_huddle,is_motion,pass_rate_off,pass_rate_def,off_xpass,def_xpass,...,cover_6_def,cover_1_def,Quarters_def,Cover-2_def,Cover-0_def,Man_def,Other_def,Zone_def,off_snaps_lost,def_snaps_lost
0,2022090800,56,1.0,6.0,False,False,NaN,NaN,0.58,0.58,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022090800,80,1.0,6.0,True,True,NaN,NaN,0.58,0.58,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022090800,101,1.0,7.0,False,True,NaN,NaN,0.58,0.58,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
merged_base['week'].unique()

array([1, 2, 3, 4, 5, 6, 7, 8])